In [47]:
# !pip install openai aiohttp nest_asyncio


import json
from IPython.display import Markdown, display
import textwrap
import asyncio
import openai
import aiohttp
import nest_asyncio

nest_asyncio.apply()
openai.api_key = 'lm-studio'

code_highlight = ", the code in response will be like this ```{program_language_name}"

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

async def fetch_completion(my_msg, code_highlights = False):
    async with aiohttp.ClientSession() as session:
        async with session.post(
            'http://localhost:1234/v1/chat/completions',
            headers={
                # 'Authorization': f'Bearer {openai.api_key}',
                'Content-Type': 'application/json'
            },
            json={
                'model': 'meta-llama-3.1-8b-instruct',
                "messages": [
                  { "role": "system", "content": "if code is in response add the code like this ```{program_language_name}" },
                  { "role": "user", "content": my_msg if not code_highlights else my_msg + code_highlight }
                ],
                'temperature': 0.7,
                "max_tokens": -1, # -1 for unlimited
                "stream": True
            }
        ) as response:
            content = ""
            display_handle = display(Markdown(content), display_id=True)
            async for line in response.content:
                line = line.decode('utf-8').strip()
                if line.startswith("data: "):
                    data = line[len("data: "):]
                    if data != '[DONE]':
                        json_data = json.loads(data)
                        delta = json_data['choices'][0]['delta']
                        if 'content' in delta:
                            content += delta['content']
                            display_handle.update(to_markdown(content))
            return content

# prompt = "Once upon a time"
# full_ans = await fetch_completion(prompt)
# print(full_ans)


In [49]:
prompt = "write in max 100 words python code that get url data"
full_ans = await fetch_completion(prompt, code_highlights=True)
# print(full_ans)

> Here's a Python script using `requests` library to fetch URL data:
> 
> ```python
> import requests
> 
> def get_url_data(url):
>     try:
>         response = requests.get(url)
>         return response.text
>     except Exception as e:
>         print(f"Error: {e}")
>         return None
> 
> url = input("Enter the URL: ")
> data = get_url_data(url)
> 
> if data is not None:
>     print(data[:100])  # Print first 100 characters of the HTML content
> ```
> 
> To use this code, replace `requests` with `pip install requests` in your terminal. This script will fetch the HTML content from the entered URL and print it. Note that the output may be too large for some URLs; you can adjust the slicing as per your needs.